In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get('DART_API_KEY')

In [2]:
from typing import *
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

session = Session()
retry = Retry(connect=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [3]:
URL = 'https://opendart.fss.or.kr/api/list.json'

PRAMS = {
    'crtfc_key': API_KEY,
    'bgn_de' : '20210104',
    'end_de' : '20210104',
    # 'pblntf_ty' : 'D',
    'pblntf_detail_ty': 'D002'
}

In [4]:
params = dict(PRAMS, **{
    'page_no': 1,
    'page_count': 1
    }
)

res = session.get(URL, params=params)
json = res.json()

page_count = 100
total_count = json['total_count']
total_page = total_count // page_count + (total_count % page_count > 0)
    
json

{'status': '000',
 'message': '정상',
 'page_no': 1,
 'page_count': 1,
 'total_count': 112,
 'total_page': 112,
 'list': [{'corp_code': '01262023',
   'corp_name': '루트락',
   'stock_code': '253610',
   'corp_cls': 'N',
   'report_nm': '임원ㆍ주요주주특정증권등소유상황보고서',
   'rcept_no': '20210104000506',
   'flr_nm': '김성구',
   'rcept_dt': '20210104',
   'rm': ''}]}

In [5]:
report_list = []

for page in range(1, total_page+1):
    params = dict(PRAMS, **{
        'page_no': page,
        'page_count': 100
        }
    )

    res = session.get(URL, params=params)
    json = res.json()
    
    report_list += json['list']

In [7]:
import pandas as pd

report_df = pd.DataFrame(report_list)
report_df.to_csv('../data/preprocessed/20210104_D002_list.csv',
                 index=False,
                 encoding='cp949'
)

In [ ]:
report_df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,01262023,루트락,253610,N,임원ㆍ주요주주특정증권등소유상황보고서,20210104000506,김성구,20210104,
1,00291860,조광ILI,044060,K,임원ㆍ주요주주특정증권등소유상황보고서,20210104000504,이브이첨단소재,20210104,
2,00927558,유바이오로직스,206650,K,[기재정정]임원ㆍ주요주주특정증권등소유상황보고서,20210104000502,전동운,20210104,
3,01262023,루트락,253610,N,임원ㆍ주요주주특정증권등소유상황보고서,20210104000497,엘앤케이바이오,20210104,
4,00190321,케이티,030200,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210104000496,김봉기,20210104,
...,...,...,...,...,...,...,...,...,...
107,00130763,서울반도체,046890,K,임원ㆍ주요주주특정증권등소유상황보고서,20210104000050,이용혁,20210104,
108,00130763,서울반도체,046890,K,임원ㆍ주요주주특정증권등소유상황보고서,20210104000048,정경재,20210104,
109,00108913,대교,019680,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210104000038,김연화,20210104,
110,00130763,서울반도체,046890,K,임원ㆍ주요주주특정증권등소유상황보고서,20210104000037,정의영,20210104,
